In [1]:
# Setup
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
#from matplotlib import style
#style.use('fivethirtyeight')
from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import AdaBoostClassifier
from sklearn.neural_network import MLPClassifier
from sklearn.model_selection import train_test_split
#from sklearn.preprocessing import LabelEncoder
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import confusion_matrix
from sklearn.model_selection import TimeSeriesSplit
from sklearn.model_selection import KFold
from sklearn import preprocessing
from sklearn.neighbors import KNeighborsClassifier
#!pip install imblearn
from imblearn.over_sampling import ADASYN


In [2]:
# Import recategorised data
wholedata = pd.read_csv("recategorised_data.csv")
wholedata.describe()

,Entero,Rain24,Rain48,Rain72,RainWA,Wdirection,Wspeed,Solarhours,BeachType,BeachDirection,Entero_level,Wspeed_level,Wdirection_level,on_offshore
count,2017.000000,2017.000000,2017.000000,2017.000000,2017.000000,2017.000000,2017.000000,2017.000000,2017.000000,2017.000000,2017.000000,2017.000000,2017.000000,2017.000000
mean,182.596926,2.052960,4.378617,6.878463,13.310040,187.879028,2.702033,6.703718,0.491324,4.221616,0.073872,1.010412,8.029251,0.869113
std,1271.550681,5.678971,9.307389,12.594966,25.209095,90.915915,1.670540,3.849683,0.500049,4.995729,0.261627,0.106303,4.073803,0.861271
min,10.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.000000,0.000000,0.000000,0.000000,0.000000
25%,10.000000,0.000000,0.000000,0.000000,0.000000,113.000000,1.500000,3.500000,0.000000,1.000000,0.000000,1.000000,5.000000,0.000000
50%,10.000000,0.000000,0.490000,1.460000,2.420000,211.000000,2.400000,7.200000,0.000000,2.000000,0.000000,1.000000,9.000000,1.000000
75%,20.000000,0.980000,4.290000,7.730000,15.160000,253.000000,3.400000,10.000000,1.000000,10.000000,0.000000,1.000000,11.000000,2.000000
max,24196.000000,57.900000,115.250000,127.320000,251.620000,360.000000,14.200000,14.100000,1.000000,15.000000,1.000000,2.000000,15.000000,2.000000


In [3]:
wholedata.head()

,DATE,Entero,Rain24,Rain48,Rain72,RainWA,Wdirection,Wspeed,Solarhours,BeachName,BeachType,BeachDirection,Entero_level,Wspeed_level,Wdirection_level,on_offshore
0,1995-12-16,10.0,9.17,13.67,37.00,59.84,200,1.5,1.8,Clarks,1,15,0,1,9,0
1,1995-12-27,10.0,0.00,0.00,0.33,0.33,230,5.7,8.5,Clarks,1,15,0,1,10,2
2,1995-12-28,10.0,2.50,2.50,2.50,7.50,320,3.6,11.7,Clarks,1,15,0,1,14,1
3,1996-01-07,10.0,0.00,0.00,0.00,0.00,130,4.1,7.9,Clarks,1,15,0,1,6,0
4,1996-01-08,10.0,0.00,0.00,0.00,0.00,100,3.6,4.4,Clarks,1,15,0,1,4,2


In [4]:
wholedata['DATE']=wholedata['DATE'].astype('datetime64')
# put aside test_set for final test
test_set = wholedata[wholedata['DATE'].dt.year>=2017]
test_set = test_set.drop(["Entero", "RainWA", "BeachName", "Wspeed", "Wdirection", "BeachDirection"], axis=1)
test_set['DATE']=test_set['DATE'].astype('datetime64')
test_set = test_set.reindex(columns=(list([c for c in test_set.columns if c != "Entero_level"]) + ["Entero_level"]))
test_set.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 105 entries, 215 to 2016
Data columns (total 10 columns):
 #   Column            Non-Null Count  Dtype         
---  ------            --------------  -----         
 0   DATE              105 non-null    datetime64[ns]
 1   Rain24            105 non-null    float64       
 2   Rain48            105 non-null    float64       
 3   Rain72            105 non-null    float64       
 4   Solarhours        105 non-null    float64       
 5   BeachType         105 non-null    int64         
 6   Wspeed_level      105 non-null    int64         
 7   Wdirection_level  105 non-null    int64         
 8   on_offshore       105 non-null    int64         
 9   Entero_level      105 non-null    int64         
dtypes: datetime64[ns](1), float64(4), int64(5)
memory usage: 9.0 KB


In [5]:
# train/validation set excluding test_set
data_frame = wholedata[wholedata['DATE'].dt.year<2017]
data_frame = data_frame[data_frame['DATE'].dt.year>=2000]
recategorised_data = data_frame.sort_values(by=['DATE'])
recategorised_data = recategorised_data.drop(["Entero", "RainWA", "BeachName", "Wspeed", "Wdirection", "BeachDirection"], axis=1)
recategorised_data['DATE']=recategorised_data['DATE'].astype('datetime64')
recategorised_data = recategorised_data.reindex(columns=(list([c for c in recategorised_data.columns if c != "Entero_level"]) + ["Entero_level"]))
recategorised_data.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1543 entries, 803 to 1997
Data columns (total 10 columns):
 #   Column            Non-Null Count  Dtype         
---  ------            --------------  -----         
 0   DATE              1543 non-null   datetime64[ns]
 1   Rain24            1543 non-null   float64       
 2   Rain48            1543 non-null   float64       
 3   Rain72            1543 non-null   float64       
 4   Solarhours        1543 non-null   float64       
 5   BeachType         1543 non-null   int64         
 6   Wspeed_level      1543 non-null   int64         
 7   Wdirection_level  1543 non-null   int64         
 8   on_offshore       1543 non-null   int64         
 9   Entero_level      1543 non-null   int64         
dtypes: datetime64[ns](1), float64(4), int64(5)
memory usage: 132.6 KB


In [6]:
# Set random state
RANDOM_STATE = 1234
np.random.seed(RANDOM_STATE)

# Extract features and label
def extract(dataframe):
    X = np.array(dataframe.iloc[:, 1:-1])
    y = np.array(dataframe.iloc[:, -1])
    return X, y

# Timeseries split
ts = TimeSeriesSplit(n_splits=5)

# Cross-validation split
kf = KFold(n_splits=4)

In [7]:
def rolling_cv(hyperparameters, fit_model):
    # Length of the test set
    l_test = 0

    # Pre-allocate space for results
    cv = np.zeros((ts.n_splits, kf.n_splits, len(hyperparameters)))

    # Multi-split the data into train sets and test sets in a timely manner
    ts_idx = -1
    for train_index, test_index in ts.split(recategorised_data):    
        ts_idx += 1
        train, test = recategorised_data.iloc[train_index, :], recategorised_data.iloc[test_index, :]
        l_test = len(test_index)

        # Cross-validate train sets to get the best hyperparameter(s)
        kf_idx = -1
        for train_index, validation_index in kf.split(train):
            kf_idx += 1
            X_train, y_train = extract(train.iloc[train_index, :])
            X_valdn, y_valdn = extract(train.iloc[validation_index, :])

            # Fit the scaler to X_train, and then use it to transform both the train set and the test set
            transfromer = preprocessing.Normalizer().fit(X_train[:, 0:3])
            X_train[:, 0:3] = transfromer.transform(X_train[:, 0:3])
            X_valdn[:, 0:3] = transfromer.transform(X_valdn[:, 0:3])

            # Oversample the train set with ADASYN
            adasyn = ADASYN(sampling_strategy="minority", n_neighbors=1)
            X_train, y_train = adasyn.fit_resample(X_train, y_train)

            # Test hyperparameters
            idx = -1
            for i in hyperparameters:
                idx += 1
                knn = fit_model(i)
                knn.fit(X_train, y_train)

                # Better safe than sorry, so the true negative rate is what we care the most - should be true positive rate
                tn, fp, fn, tp = confusion_matrix(y_valdn, knn.predict(X_valdn)).ravel()
                cv[ts_idx, kf_idx, idx] = tp/(tp+fn)
    return l_test, cv

In [8]:
def walking_eval(l_test, fitted_model):
    r = []
    # Evaluation by walking backwards, i.e. remove 6 obesersations from the test set at one step until it runs out
    for i in range(0, l_test, 6):
        train = recategorised_data.iloc[0:-(l_test+i), :]
        test = recategorised_data.iloc[-(l_test+i):, :]

        X_train, y_train = extract(train)
        X_test, y_test = extract(test)

        transfromer = preprocessing.Normalizer().fit(X_train[:, 0:3])
        X_train[:, 0:3] = transfromer.transform(X_train[:, 0:3])
        X_test[:, 0:3] = transfromer.transform(X_test[:, 0:3])

        adasyn = ADASYN(sampling_strategy="minority", n_neighbors=1)
        X_train, y_train = adasyn.fit_resample(X_train, y_train)

        # Plug in the best hyperparameter(s) from cv
        knn = fitted_model
        knn.fit(X_train, y_train)

        tn, fp, fn, tp = confusion_matrix(y_test, knn.predict(X_test)).ravel()
        r.append(tp/(tp+fn))
    return np.array(r)

In [9]:
#KNN
# Train
neighbors = range(1, 10, 1)
def knn(i):
    knn = KNeighborsClassifier(n_neighbors=i)
    return knn
l_test, cv = rolling_cv(neighbors, knn)

means = np.mean(cv, axis=(0,1))
best = neighbors[np.argmax(means)]
print("The best KNN is with %s neighbour(s)" %(best))

# Evaluate
fitted_model = knn(best)
r = walking_eval(l_test, fitted_model)

print("The best KNN is with an average sensitivity of %s" %(np.mean(r)))


The best KNN is with 9 neighbour(s)
The best KNN is with an average sensitivity of 0.396903274729783


In [10]:
# Final KNN model test on separate test-set
test_set_data = test_set.iloc[:,1:-1]
#test_set_data = sc.transform(test_set_data)
test_set_labels = test_set.iloc[:,-1]

print("Accuracy score")
print(fitted_model.score(test_set_data,test_set_labels))
print("Confusion matrix")
print(confusion_matrix(test_set_labels,fitted_model.predict(test_set_data)))
print("Targets")
print(test_set_labels)
print("Predictions")
print(fitted_model.predict(test_set_data))

Accuracy score
0.6285714285714286
Confusion matrix
[[59 29]
 [10  7]]
Targets
215     0
216     0
217     0
218     0
219     0
       ..
2012    0
2013    0
2014    0
2015    0
2016    0
Name: Entero_level, Length: 105, dtype: int64
Predictions
[0 0 1 1 0 1 1 1 0 0 0 0 0 1 0 0 1 0 0 1 0 0 0 0 1 1 1 0 1 0 1 0 0 0 0 0 1
 0 0 0 0 0 1 0 0 1 0 0 0 0 0 1 0 0 0 1 0 0 1 0 0 0 0 1 0 0 0 1 1 0 1 1 0 1
 1 0 0 1 0 1 0 1 1 1 1 1 0 0 0 1 0 0 0 0 0 0 0 0 1 0 0 1 0 0 0]


In [11]:
# BDT
# Train
learning_rates = np.linspace(0.1, 1, 9)
def ada(i):
    ada = AdaBoostClassifier(learning_rate=i)
    return ada
l_test, cv = rolling_cv(learning_rates, ada)

means = np.mean(cv, axis=(0,1))
best = learning_rates[np.argmax(means)]
print("The best BDT is with a learning rate of %s" %(best))

# Evaluate
fitted_model = ada(best)
r = walking_eval(l_test, fitted_model)

print("The best BDT is with an average sensitivity of %s" %(np.mean(r)))

The best BDT is with a learning rate of 0.1
The best BDT is with an average sensitivity of 0.4294089214118263


In [12]:
# Final BDT model test on separate test-set
test_set_data = test_set.iloc[:,1:-1]
#test_set_data = sc.transform(test_set_data)
test_set_labels = test_set.iloc[:,-1]

print("Accuracy score")
print(fitted_model.score(test_set_data,test_set_labels))
print("Confusion matrix")
print(confusion_matrix(test_set_labels,fitted_model.predict(test_set_data)))
print("Targets")
print(test_set_labels)
print("Predictions")
print(fitted_model.predict(test_set_data))

Accuracy score
0.6761904761904762
Confusion matrix
[[70 18]
 [16  1]]
Targets
215     0
216     0
217     0
218     0
219     0
       ..
2012    0
2013    0
2014    0
2015    0
2016    0
Name: Entero_level, Length: 105, dtype: int64
Predictions
[0 1 0 1 1 0 1 1 0 0 1 0 1 1 0 1 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 1 0 0 0 0 0 0 0 0 0 0 0 1 1 1
 1 1 0 1 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]


In [13]:
# ANN
# Skip the training part as the data set is not large enough to tune the hyperparameters properly for ANN

# For the completeness of the experiment design, evaluate ANN on a setting off the top of my head
ann = MLPClassifier(solver="sgd", learning_rate="adaptive", hidden_layer_sizes=(4, 2), max_iter=1000) 
r = walking_eval(ann)
print("The average sensitivity for the ANNs is %s" %(np.mean(r)))

TypeError: walking_eval() missing 1 required positional argument: 'fitted_model'